# Импортирую библиотеки

In [144]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import gpxpy
import os
from shapely.geometry import box, LineString
import matplotlib.pyplot as plt
import contextily as ctx
import time
import psycopg2
import cv2
import time
import math
import datetime
import concurrent.futures
import requests
from typing import List, Dict

# Создаю список с ссылками для скачивания

In [145]:
links = []

with open("./Links.txt", mode="r", encoding="UTF-8") as f:
    for i in f:
        links.append(i.strip())

In [146]:
links

['https://caucasia.ru/gpx/8EWQAV62.gpx',
 'https://caucasia.ru/gpx/CPOQQN9R.gpx',
 'https://caucasia.ru/gpx/CVBOS4W2.gpx',
 'https://caucasia.ru/gpx/HTGT4UOT.gpx',
 'https://caucasia.ru/gpx/HQUWT5UB.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/XKSRMH5L.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/G6I1K7J9.gpx']

# Загружаю gpx файлы из ссылок


In [147]:
for num, url in enumerate(links):
    try:
        response = requests.get(url)
        filename = f"track{num}.gpx"
        
        with open(f"data/gpx/{filename}", mode="wb") as f:
            f.write(response.content)
    except Exception as e:
        print("Ошибка при скачивании")


In [148]:
gpx_list = os.listdir("data/gpx")

In [149]:
gpx_list

['track0.gpx',
 'track2.gpx',
 'track7.gpx',
 'track3.gpx',
 'track4.gpx',
 'track8.gpx',
 'track1.gpx',
 'track6.gpx',
 'track5.gpx']

# Создаю изображения карты с маршрутом по трекам

In [150]:
margin = 0.02
result = []

for file in gpx_list:
    with open(f"{"data/gpx"}/{file}", mode="r", encoding="UTF-8") as f:
        gpx = gpxpy.parse(f)
    
    lats, lons, alts = [], [], []

    time_list = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lats.append(point.latitude)
                lons.append(point.longitude)
                alts.append(point.elevation)
                result.append({
                        "track_id": file,
                        "analysis_date": point.time.date(),
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "altitude": point.elevation})
                
    bbox = box(
            min(lons) - margin,
            min(lats) - margin,
            max(lons) + margin,
            max(lats) + margin
        )
    track_line = LineString(zip(lons, lats))

    gdf_bbox = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")
    gdf_bbox_web = gdf_bbox.to_crs(epsg=3857)

    gdf_track = gpd.GeoDataFrame(geometry=[track_line], crs="EPSG:4326")
    gdf_track_web = gdf_track.to_crs(epsg=3857)

    _, ax = plt.subplots(figsize=(10, 8))

    gdf_bbox_web.plot(ax=ax, alpha=0)
    gdf_track_web.plot(ax=ax, color="red", linewidth=2)

    ctx.add_basemap(ax, crs=gdf_bbox_web.crs, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()
    os.makedirs("data/image", exist_ok=True)

    plt.savefig(f"{"data/image"}/{file[:-4]}", dpi=150, bbox_inches="tight", pad_inches=0)
    plt.close()


In [151]:
from geopy.distance import geodesic

def distance(lat1, lon1, lat2, lon2):
    """
    Рассчитывает расстояние между двумя точками на Земле
    """
    
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    
    distance = geodesic(point1, point2).kilometers
    return distance

In [152]:
names_json = []
ll = list(zip(lats, lons))

In [153]:
for lat, lon in (ll[0], ll[len(ll)//5],  ll[len(ll)//2], ll[len(ll)//3],  ll[-1]):
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json", headers=headers)
            print(response)
            json = response.json()
            time.sleep(1.5)

            names_json.append(json["address"]["county"])
        except Exception as e:
            print(f"Возникла ошибка при получении данных с API: {e}")

<Response [403]>
Возникла ошибка при получении данных с API: Expecting value: line 1 column 1 (char 0)
<Response [403]>
Возникла ошибка при получении данных с API: Expecting value: line 1 column 1 (char 0)
<Response [200]>
<Response [200]>
<Response [200]>


In [154]:
names_json

['Бахчисарайский район', 'Бахчисарайский район', 'Бахчисарайский район']

In [155]:
result

[{'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.58919885044738,
  'longitude': 38.40022087097168,
  'altitude': 187.92631959139499},
 {'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.58916,
  'longitude': 38.40019,
  'altitude': 188.1595839999934},
 {'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.58923,
  'longitude': 38.40004,
  'altitude': 188.61648000000963},
 {'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.58956,
  'longitude': 38.39969,
  'altitude': 189.50649600000997},
 {'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.59005,
  'longitude': 38.39912,
  'altitude': 188.22463999998544},
 {'track_id': 'track0.gpx',
  'analysis_date': datetime.date(2025, 11, 17),
  'latitude': 44.59011,
  'longitude': 38.39904,
  'altitude': 188.56793600000492},
 {'track_id': 'track0.g

# Создаю базу данных

In [156]:
def connect_db():
     return psycopg2.connect(
        database="db_arthur", 
        user="arthur", 
        password="146a",
        host="localhost",
        port=5430)

In [157]:
try:
    cursor = connect_db().cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS track_analysis (
            id SERIAL PRIMARY KEY,
            track_id VARCHAR,
            analysis_date DATE,
            region VARCHAR,
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            altitude DOUBLE PRECISION,
            step_frequency DOUBLE PRECISION,
            temperature DOUBLE PRECISION,
            terrain_type VARCHAR,
            key_objects VARCHAR,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    for i in result:
        cursor.execute("""
            INSERT INTO track_analysis
            (track_id, analysis_date, latitude, longitude, altitude)
            VALUES (%s, %s, %s, %s, %s)
        """,
            (i["track_id"],
            i["analysis_date"],
            i["latitude"],
            i["longitude"],
            i["altitude"]))

    connect_db().commit()
except Exception as e:
    print(f"Ошибка при создании базы данных: {e}")
finally:
    if connect_db():
        cursor.close()
        connect_db().close()

In [ ]:
df_test = pd.DataFrame(result)

,track_id,analysis_date,latitude,longitude,altitude
0,track0.gpx,2025-11-17,44.589199,38.400221,187.926320
1,track0.gpx,2025-11-17,44.589160,38.400190,188.159584
2,track0.gpx,2025-11-17,44.589230,38.400040,188.616480
3,track0.gpx,2025-11-17,44.589560,38.399690,189.506496
4,track0.gpx,2025-11-17,44.590050,38.399120,188.224640
...,...,...,...,...,...
32001,track5.gpx,2021-01-05,44.550820,33.899888,311.700000
32002,track5.gpx,2021-01-05,44.551270,33.899987,310.180000
32003,track5.gpx,2021-01-05,44.551492,33.899950,311.090000
32004,track5.gpx,2021-01-05,44.551735,33.899732,313.530000


# Получаю температуру

In [ ]:
def temp(lat, lon, date):
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        'latitude': lat,        # Ширина
        'longitude': lon,       # Долгота
        'start_date': date,
        'end_date': date,
        'hourly': 'temperature_2m',
        "timezone":"auto"
    }
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0'}

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    if response.status_code == 200:
        data = response.json()
        return data["hourly"]['temperature_2m'][12]       
    else:
        print(f"Данные не получены")

In [166]:
df_test.shape

(6730, 12)

In [ ]:
def analysis_wather():
    '''
    есть 4 айдишника и ноль, где между нулем и след айдишниками равная длина
    изначально вызвать апишку для этих 5 точек
    надо проходить по всему дф, и если айди совпадает с тем, которое у нас есть (с левым) 
    то берем подсчитанное значение, и менять левое значение индекса на следующее в списке, 
    а если не совпадает, то искать по формуле интерполяции 
    '''

    
    n = df_test.shape[0]
    key_index = [0, n//4, n//2, n*3//4, -1]

    temperatures = {x:temp(x)['temperature'] for x in key_index}


    left, right = 0, 1
    for idx, point in df_test.iloc[::5].iterrows():
        if right < 4 and idx >= key_index[right]:
            left += 1
            right += 1
    if idx == key_index[left]:
                    temperature = temperatures[idx]
                    
    else:
        temperature = temperatures[key_index[left]] + (temperatures[key_index[right]] - temperatures[key_index[left]]) * \
                        (idx - key_index[left]) / (key_index[right]-key_index[left])
    
    df_test = df_test["temperatures"] = temperature
    return df_test
    

    
    


In [163]:
temp(44.539757, 33.882218, "2021-01-05")

HTTPError: 429 Client Error: Too Many Requests for url: https://archive-api.open-meteo.com/v1/archive?latitude=44.539757&longitude=33.882218&start_date=2021-01-05&end_date=2021-01-05&hourly=temperature_2m&timezone=auto

In [ ]:
def get_waterher(result):
    """
    Функция записывающая каждую пятую точку погоды в result
    (доработать)
    """

    for idx, point in enumerate(result):
        lat = point["latitude"]
        lon = point["longitude"]
        date = point["analysis_date"]
        
        # Фиксируем только каждую пятую точку
        if idx % 5 == 0:
            temp(lat, lon, date)
            print(idx)

In [ ]:


track_data = df_test[df_test['main_id'] == 8]

key_index = (0, track_data.shape[0]//4, track_data.shape[0]//2, track_data.shape[0]//4 * 3, -1,)

KeyError: 'main_id'

добоавляю в базу данных температуру

In [ ]:
try:
    cursor = connect_db().cursor()

    for i in result:
        cursor.execute("""
            INSERT INTO track_analysis
            (temperature)
            VALUES (%s)
        """,
            (i["temperatures"]))

    connect_db.commit()
except Exception as e:
    print(f"Ошибка при создании базы данных: {e}")
finally:
    if connect_db:
        cursor.close()
        connect_db.close()

Ошибка при создании базы данных: 'temperatures'
